In [1]:
pip install numpy scipy librosa scikit-learn sounddevice

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import pickle
import numpy as np
import sounddevice as sd
from scipy.io.wavfile import write
import librosa
from sklearn.mixture import GaussianMixture
import time

class VoiceAuthenticator:
    def __init__(self, models_dir="voice_models"):
        self.fs = 22050  # Sample rate
        self.seconds = 3  # Recording duration
        self.models_dir = models_dir
        
        # Create models directory if it doesn't exist
        if not os.path.exists(models_dir):
            os.makedirs(models_dir)
    
    def extract_features(self, audio_data, sample_rate):
        """Extract MFCCs from audio data"""
        try:
            # Extract 20 MFCC features
            mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=20)
            
            # Transpose to get time-based frames for GMM
            return mfccs.T
        except Exception as e:
            print(f"Error extracting features: {e}")
            return None
    
    def record_audio(self, filename="temp_recording.wav"):
        """Record audio from microphone"""
        print(f"Recording for {self.seconds} seconds...")
        
        try:
            # Record audio
            recording = sd.rec(int(self.seconds * self.fs), samplerate=self.fs, channels=1)
            sd.wait()  # Wait until recording is finished
            
            print("Recording finished")
            
            # Save as WAV file
            write(filename, self.fs, recording)
            
            # Convert to mono and return
            audio_data = recording.flatten()
            return audio_data
        except Exception as e:
            print(f"Error recording audio: {e}")
            return None
    
    def enroll_user(self, username):
        """Enroll a new user by creating a voice model"""
        # Check if username already exists
        model_path = os.path.join(self.models_dir, f"{username}.gmm")
        if os.path.exists(model_path):
            overwrite = input(f"User {username} already exists. Overwrite? (y/n): ").lower()
            if overwrite != 'y':
                print("Enrollment cancelled.")
                return False
        
        print(f"Enrolling user: {username}")
        print("Please speak for voice enrollment when prompted.")
        time.sleep(1)
        
        features_list = []
        
        # Record multiple samples for better model
        for i in range(3):
            print(f"\nSample {i+1}/3. ", end="")
            input("Press Enter and start speaking...")
            audio_data = self.record_audio()
            
            if audio_data is None:
                print("Failed to record audio. Please try again.")
                return False
                
            features = self.extract_features(audio_data, self.fs)
            
            if features is None:
                print("Failed to extract features. Please try again.")
                return False
                
            features_list.append(features)
        
        # Concatenate all features
        all_features = np.vstack(features_list)
        
        # Check if we have enough data
        if len(all_features) < 20:
            print("Not enough voice data collected. Please try again and speak clearly.")
            return False
        
        try:
            # Train a Gaussian Mixture Model
            gmm = GaussianMixture(n_components=16, covariance_type='diag', random_state=42)
            gmm.fit(all_features)
            
            # Save the model
            with open(model_path, 'wb') as f:
                pickle.dump(gmm, f)
            
            print(f"\nEnrollment complete for {username}!")
            return True
        except Exception as e:
            print(f"Error during model training: {e}")
            return False
    
    def authenticate_user(self, username):
        """Authenticate a user against their stored voice model"""
        model_path = os.path.join(self.models_dir, f"{username}.gmm")
        
        # Check if user exists
        if not os.path.exists(model_path):
            print(f"User {username} not found!")
            return False
        
        try:
            # Load the GMM model
            with open(model_path, 'rb') as f:
                gmm = pickle.load(f)
            
            print(f"Authenticating user: {username}")
            input("Press Enter and speak for authentication...")
            
            # Record authentication attempt
            audio_data = self.record_audio()
            
            if audio_data is None:
                print("Failed to record audio. Please try again.")
                return False
                
            features = self.extract_features(audio_data, self.fs)
            
            if features is None or len(features) == 0:
                print("Failed to extract features. Please try again.")
                return False
            
            # Calculate log likelihood score
            score = gmm.score(features)
            threshold = -35  # This threshold needs tuning based on your environment
            
            print(f"Score: {score:.2f}, Threshold: {threshold}")
            
            if score >= threshold:
                print(f"Authentication successful! Welcome, {username}.")
                return True
            else:
                print("Authentication failed! Voice does not match.")
                return False
        except Exception as e:
            print(f"Error during authentication: {e}")
            return False
    
    def list_users(self):
        """List all enrolled users"""
        try:
            users = []
            if os.path.exists(self.models_dir):
                for file in os.listdir(self.models_dir):
                    if file.endswith(".gmm"):
                        users.append(file.replace(".gmm", ""))
            
            if users:
                print("Enrolled users:")
                for user in users:
                    print(f"- {user}")
            else:
                print("No users enrolled yet.")
            
            return users
        except Exception as e:
            print(f"Error listing users: {e}")
            return []


def main():
    try:
        authenticator = VoiceAuthenticator()
        
        while True:
            print("\n===== Voice Authentication System =====")
            print("1. Enroll New User")
            print("2. Authenticate User")
            print("3. List Users")
            print("4. Exit")
            
            choice = input("Enter your choice (1-4): ")
            
            if choice == "1":
                username = input("Enter username for enrollment: ").strip()
                if not username:
                    print("Username cannot be empty.")
                    continue
                authenticator.enroll_user(username)
            
            elif choice == "2":
                username = input("Enter username to authenticate: ").strip()
                if not username:
                    print("Username cannot be empty.")
                    continue
                authenticator.authenticate_user(username)
            
            elif choice == "3":
                authenticator.list_users()
            
            elif choice == "4":
                print("Exiting...")
                break
            
            else:
                print("Invalid choice. Please try again.")
    except KeyboardInterrupt:
        print("\nProgram interrupted. Exiting...")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


if __name__ == "__main__":
    main()


===== Voice Authentication System =====
1. Enroll New User
2. Authenticate User
3. List Users
4. Exit


Enter your choice (1-4):  1
Enter username for enrollment:  nuha
User nuha already exists. Overwrite? (y/n):  zarin


Enrollment cancelled.

===== Voice Authentication System =====
1. Enroll New User
2. Authenticate User
3. List Users
4. Exit


Enter your choice (1-4):  2
Enter username to authenticate:  nuha


Authenticating user: nuha


...

Authenticating user: nuha


Press Enter and speak for authentication... 


Recording for 3 seconds...
Recording finished
Score: -91.01, Threshold: -35
Authentication failed! Voice does not match.

===== Voice Authentication System =====
1. Enroll New User
2. Authenticate User
3. List Users
4. Exit


Enter your choice (1-4):  3


Enrolled users:
- nuha

===== Voice Authentication System =====
1. Enroll New User
2. Authenticate User
3. List Users
4. Exit
